<a href="https://colab.research.google.com/github/BryanBM8/Rupiaj/blob/main/ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report


In [ ]:
names = ['1 RIBU ASLI', '1 RIBU PALSU', '10 RIBU ASLI', '10 RIBU PALSU',
         '100 RIBU ASLI', '100 RIBU PALSU', '20 RIBU ASLI', '20 RIBU PALSU',
         '5 RIBU ASLI', '5 RIBU PALSU', '50 RIBU ASLI', '50 RIBU PALSU']

In [ ]:
def augment_image(image):
    augmented_images = []

    # Rotasi
    for angle in [-30, 30]:
        rotation_matrix = cv2.getRotationMatrix2D((image.shape[1] // 2, image.shape[0] // 2), angle, 1.0)
        rotated = cv2.warpAffine(image, rotation_matrix, (image.shape[1], image.shape[0]))
        augmented_images.append(rotated)

    # Ubah kecerahan
    for brightness_factor in [0.8, 1.0, 1.2]:
        hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        hsv[:, :, 2] = np.clip(hsv[:, :, 2] * brightness_factor, 0, 255).astype(np.uint8)
        bright_img = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
        augmented_images.append(bright_img)

    # Tambah noise
    for _ in range(2):
        noise = np.random.normal(0, 15, image.shape).astype(np.uint8)
        noisy_img = cv2.add(image, noise)
        augmented_images.append(noisy_img)

    return augmented_images


# microtext
def extract_microtext(image_gray):
    edges = cv2.Canny(image_gray, 50, 150)
    return np.sum(edges) / edges.size

# warna
def extract_color(image):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hue_mean = np.mean(hsv[:, :, 0])
    sat_mean = np.mean(hsv[:, :, 1])
    val_mean = np.mean(hsv[:, :, 2])
    return hue_mean, sat_mean, val_mean

# benang pengaman
def detect_thread(image_gray):
    _, thresh = cv2.threshold(image_gray, 128, 255, cv2.THRESH_BINARY)
    vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 15))
    thread = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel)
    return np.sum(thread) / thread.size


def extract_features(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray=cv2.equalizeHist(gray)
    microtext_density = extract_microtext(gray)
    hue, saturation, value = extract_color(image)
    thread_density = detect_thread(gray)
    return [ microtext_density, hue, saturation, value, thread_density]


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
dataset_path = "/content/drive/MyDrive/CompVis Projek/UANGUV"

def load_dataset_with_folder_structure(folder_path):
    images_folder = os.path.join(folder_path, "images")
    labels_folder = os.path.join(folder_path, "labels")

    data = []
    labels = []
    count=0

    for img_name in os.listdir(images_folder):
        if img_name.endswith(".jpg"):
            img_path = os.path.join(images_folder, img_name)
            label_path = os.path.join(labels_folder, os.path.splitext(img_name)[0] + ".txt")  # File label

            img = cv2.imread(img_path)
            if img is not None and os.path.exists(label_path):  # Pastikan gambar dan label ada
                resized_img = cv2.resize(img, (512, 512))  # Resize gambar
                features = extract_features(resized_img)  # Ekstraksi fitur
                data.append(features)

                with open(label_path, "r") as f:
                    first_line = f.readline().strip()  # Ambil baris pertama
                    class_index = int(first_line.split()[0])  # Digit pertama sebagai label
                    labels.append(class_index)

                augmented_imgs = augment_image(resized_img)
                for aug_img in augmented_imgs:
                    aug_features = extract_features(aug_img)
                    data.append(aug_features)
                    labels.append(class_index)


    return np.array(data, dtype=np.float32), np.array(labels, dtype=np.int32)



In [ ]:
x_train, y_train= load_dataset_with_folder_structure(dataset_path+'/train/')
from sklearn.utils import shuffle

x_train, y_train= shuffle(x_train, y_train, random_state=42)
x_test, y_test=load_dataset_with_folder_structure(dataset_path+'/test/')
x_test, y_test=shuffle(x_test, y_test, random_state=42)

In [ ]:
def balance_dataset_undersample(data, labels):
    unique_labels, counts = np.unique(labels, return_counts=True)
    min_count = min(counts)

    balanced_data = []
    balanced_labels = []

    label_to_data = {label: [] for label in unique_labels}
    for d, l in zip(data, labels):
        label_to_data[l].append(d)

    for label in unique_labels:
        samples = label_to_data[label][:min_count]
        balanced_data.extend(samples)
        balanced_labels.extend([label] * min_count)

    return np.array(balanced_data, dtype=np.float32), np.array(balanced_labels, dtype=np.int32)


x_train, y_train= balance_dataset_undersample(x_train, y_train)

In [ ]:
unique_labels, counts = np.unique(y_train, return_counts=True)

print("Jumlah data per label:")
for label, count in zip(unique_labels, counts):
    print(f"Label {label}: {count} data")

Jumlah data per label:
Label 0: 2304 data
Label 1: 2304 data
Label 2: 2304 data
Label 3: 2304 data
Label 4: 2304 data
Label 5: 2304 data
Label 6: 2304 data
Label 7: 2304 data
Label 8: 2304 data
Label 9: 2304 data
Label 10: 2304 data
Label 11: 2304 data


In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

param_grid = {
    'C': [0.1],
    'gamma': [1],
    'kernel': ['rbf']
}

grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=2, cv=5)
grid.fit(x_train, y_train)

print("Best Parameters:", grid.best_params_)
print("Best Cross-Validation Accuracy:", grid.best_score_)

best_svm = grid.best_estimator_
print("Test Accuracy:", best_svm.score(x_test, y_test))


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=  59.4s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=  58.6s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=  58.7s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=  58.6s
[CV] C=0.1, gamma=1, kernel=rbf ......................................
[CV] ....................... C=0.1, gamma=1, kernel=rbf, total=  60.0s
Best Parameters: {'C': 0.1, 'gamma': 1, 'kernel': 'rbf'}
Best Cross-Validation Accuracy: 0.3797744066547682
Test Accuracy: 0.8928571428571429


In [ ]:
y_pred_svm = best_svm.predict(x_test)
print("KNN Classification Report:")
print(classification_report(y_test, y_pred_svm))

KNN Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.93      0.95       288
           1       1.00      0.77      0.87        96
           2       1.00      0.89      0.94       232
           3       1.00      0.96      0.98       104
           4       0.99      0.93      0.96       344
           5       0.35      1.00      0.52       120
           6       1.00      0.79      0.88       120
           7       1.00      0.93      0.96       144
           8       1.00      0.87      0.93       120
           9       1.00      0.94      0.97       232
          10       1.00      0.80      0.89       248
          11       1.00      0.82      0.90       136

    accuracy                           0.89      2184
   macro avg       0.94      0.89      0.90      2184
weighted avg       0.96      0.89      0.91      2184



In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_neighbors': range(1, 21),
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan']
}

grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5)
grid_search.fit(x_train, y_train)

print("Best parameters:", grid_search.best_params_)
print("Best cross-validated score:", grid_search.best_score_)

y_pred_knn = grid_search.predict(x_test)
print("KNN Classification Report:")
print(classification_report(y_test, y_pred_knn))


Best parameters: {'metric': 'manhattan', 'n_neighbors': 1, 'weights': 'uniform'}
Best cross-validated score: 0.9228875333315673
KNN Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.96      0.97       288
           1       0.89      0.92      0.90        96
           2       0.99      1.00      0.99       232
           3       1.00      1.00      1.00       104
           4       1.00      0.99      0.99       344
           5       1.00      1.00      1.00       120
           6       0.96      1.00      0.98       120
           7       1.00      1.00      1.00       144
           8       0.96      0.99      0.98       120
           9       1.00      0.98      0.99       232
          10       0.99      0.98      0.99       248
          11       1.00      1.00      1.00       136

    accuracy                           0.98      2184
   macro avg       0.98      0.98      0.98      2184
weighted avg       0.98      0.98

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(x_train, y_train)
rf.score(x_test, y_test)

y_pred_rf = rf.predict(x_test)
print("SVM Classification Report:")
print(classification_report(y_test, y_pred_rf))



SVM Classification Report:
              precision    recall  f1-score   support

           0       0.97      1.00      0.98       288
           1       1.00      0.92      0.96        96
           2       0.99      1.00      0.99       232
           3       0.98      1.00      0.99       104
           4       1.00      0.98      0.99       344
           5       1.00      1.00      1.00       120
           6       0.97      1.00      0.98       120
           7       1.00      1.00      1.00       144
           8       1.00      0.97      0.99       120
           9       1.00      0.99      1.00       232
          10       0.98      0.98      0.98       248
          11       1.00      1.00      1.00       136

    accuracy                           0.99      2184
   macro avg       0.99      0.99      0.99      2184
weighted avg       0.99      0.99      0.99      2184



In [ ]:
# from sklearn.externals import
import pickle

# pickle.dump(rf, 'rf_model.pkl', 'rb')
with open('rf_model.pkl', 'wb') as file:
    pickle.dump(rf, file)

# pickle.dump(grid_search, 'knn_model.pkl', 'rb')
with open('knn_model.pkl', 'wb') as file:
    pickle.dump(grid_search, file)

# pickle.dump(best_svm, 'svm_model.pkl', 'rb')
with open('svm_model.pkl', 'wb') as file:
    pickle.dump(best_svm, file)

In [ ]:
test_img_path = "./PROJECT DATA UANG.v3i.yolov11/valid/images/WIN_20240626_16_38_51_Pro_mp4-0000_jpg.rf.e63359b40e200d0dfed497544f92d64e.jpg"
test_img = cv2.imread(test_img_path)
if test_img is not None:
    resized_test_img = cv2.resize(test_img, (512, 512))
    test_features = extract_features(resized_test_img)
    test_features = np.array(test_features).reshape(1, -1)

    svm_result = best_svm.predict(test_features)[0]
    knn_result = grid_search.predict(test_features)[0]
    rf_result=rf.predict(test_features)[0]

    print("Prediksi SVM:", names[svm_result])
    print("Prediksi RF:", names[rf_result])
    print("Prediksi KNN:", names[knn_result])


Prediksi SVM: 100 RIBU PALSU
Prediksi RF: 100 RIBU ASLI
Prediksi KNN: 10 RIBU ASLI


In [ ]:
# import numpy as np
# import tensorflow as tf
# import os
# from sklearn.model_selection import train_test_split
# from tensorflow.keras.preprocessing.image import load_img, img_to_array
# from tensorflow.keras import layers, models, optimizers

In [ ]:
# from tensorflow.keras.applications import ResNet50
# from tensorflow.keras.models import Model

# # Definisikan model ResNet50 sebagai backbone
# base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(512, 512, 3))

# # Tambahkan custom layers di atas backbone
# model = models.Sequential([
#     base_model,  # ResNet50 sebagai backbone
#     layers.GlobalAveragePooling2D(),  # Global average pooling untuk meratakan output ResNet50
#     layers.Dense(units=4096, activation="relu"),  # Fully connected layer pertama
#     layers.Dense(units=4096, activation="relu"),  # Fully connected layer kedua
#     layers.Dense(units=1000, activation="softmax")  # Output layer untuk 1000 kelas
# ])

In [ ]:
# model.compile(optimizer='adam',
#               loss='sparse_categorical_crossentropy',
#               metrics=['accuracy'])


# model.summary()

In [ ]:
# x_val, y_val=load_dataset_with_folder_structure(dataset_path+'/valid/')
# x_val, y_val=shuffle(x_val, y_val, random_state=42)


# early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
# history = model.fit(x_train,y_train,epochs=100,validation_data=(x_val,y_val), batch_size=32)


In [ ]:
# loss, accuracy = model.evaluate(x_test,y_test)
# print(f'loss = {loss}')
# print(f'accuracy = {accuracy}')

In [ ]:
# cnn_result=model.predict(x_test)
# predicted_class_indices = np.argmax(cnn_result, axis=-1)


# correct_count = np.sum(predicted_class_indices == y_test)
# accuracy = correct_count / len(y_test)
# for i, (predicted_idx, true_idx) in enumerate(zip(predicted_class_indices, y_test)):
#     print(f"Sample {i}: Prediksi = {names[predicted_idx]}, Asli = {names[true_idx]}")


In [ ]:
# model.save('vgg16_model.h5')
